<a href="https://www.kaggle.com/code/shamitsaha/notebook5cea1b099b?scriptVersionId=260194797" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-dataset/Titanic-Dataset.csv


In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer , KNNImputer
from xgboost import XGBClassifier

In [3]:
df=pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df["Title"] = df["Name"].str.extract(" ([A-Za-z]+)\.", expand=False)
df['Title'] = df['Title'].replace(['Don','Rev','Dr','Major','Col','Capt','Countess','Lady','Jonkheer','Sir'],'Rare')
df['cabin_letter'] = df['Cabin'].astype(str).str[0].replace('n','Unknown')
df.drop(['Name','PassengerId','Ticket','Cabin'],axis =1,inplace=True)

In [5]:
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,cabin_letter
0,0,3,male,22.0,1,0,7.2500,S,Mr,Unknown
1,1,1,female,38.0,1,0,71.2833,C,Mrs,C
2,1,3,female,26.0,0,0,7.9250,S,Miss,Unknown
3,1,1,female,35.0,1,0,53.1000,S,Mrs,C
4,0,3,male,35.0,0,0,8.0500,S,Mr,Unknown
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Rare,Unknown
887,1,1,female,19.0,0,0,30.0000,S,Miss,B
888,0,3,female,NaN,1,2,23.4500,S,Miss,Unknown
889,1,1,male,26.0,0,0,30.0000,C,Mr,C


In [6]:
df.isnull().sum()

Survived          0
Pclass            0
Sex               0
Age             177
SibSp             0
Parch             0
Fare              0
Embarked          2
Title             0
cabin_letter      0
dtype: int64

In [7]:
y=df.iloc[:,0]
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [8]:
x=df.drop(['Survived',],axis=1)

In [9]:
print(x.shape)   # features
print(y.shape)   # target

(891, 9)
(891,)


In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3 , random_state =4)

In [11]:
num_col = ['Pclass','Age','SibSp','Parch','Fare']
cat_col = ['Sex','Embarked','Title','cabin_letter']

In [12]:
transformer = ColumnTransformer(
    transformers=[
      ('num' ,Pipeline([
          ('imputer',KNNImputer(n_neighbors=5)),
          ('scaler',MinMaxScaler())
      ]),num_col ),
        
      ( 'cat' , Pipeline ([
           ('imputer',SimpleImputer(strategy='most_frequent')),
          ('ohe', OneHotEncoder(handle_unknown='ignore',drop='first'))

          ] ),cat_col )], remainder='drop' 
)

# MODEL USING LogisticRegression 

In [13]:
model_lr = Pipeline([
    ('lr_pro',transformer),
    ('lr',LogisticRegression())
])

In [14]:
model_lr.fit(x_train,y_train)

Pipeline(steps=[('lr_pro',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked', 'Title',
                                                   'cabin_letter'])])),
                ('lr', LogisticRegression())])

In [15]:
y_pred_lr=model_lr.predict(x_test)

In [16]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy*100)

Accuracy: 86.19402985074626


# MODEL USING XGBoost 

In [17]:
model_xgboost = Pipeline ([
    ('pro',transformer ),
    ('classifier', XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    ))
])

In [18]:
model_xgboost.fit(x_train,y_train)

Pipeline(steps=[('pro',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked', 'Title',
                                                   'cabin_letter']...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=4, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [19]:
y_pred_xgb=model_xgboost.predict(x_test)

In [20]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred_xgb)
print("Accuracy:", accuracy*100)

Accuracy: 85.82089552238806


# this is using random forest


In [21]:
from sklearn.ensemble import RandomForestClassifier


In [22]:
model_randomforest = Pipeline ([
    ('pro',transformer ),
    ('classifier', RandomForestClassifier(
        n_estimators=300,
        max_depth=4,
        random_state=42
    ))
])

In [23]:
model_randomforest.fit(x_train,y_train)

Pipeline(steps=[('pro',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked', 'Title',
                                                   'cabin_letter'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=4, n_estimators=300,
                                        random_state=42))])

In [24]:
y_pred_rf=model_randomforest.predict(x_test)

In [25]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy*100)

Accuracy: 85.44776119402985
